# Email-Eu-core Network

Test on the *email-Eu-core network* from SNAP. The network was generated using email data from a large European research institution. The dataset contains "ground-truth" community memberships of the nodes. Each individual belongs to exactly one of 42 departments at the research institute.

See https://snap.stanford.edu/data/email-Eu-core.html

In [1]:
import networkx as nx
from networkx.algorithms import community
import numpy as np

In [2]:
data_inpath = "../../data/email-eu-core/email-eu-core.edges"
comm_inpath = "../../data/email-eu-core/email-eu-core.comm"

### Data Loading

In [3]:
def read_communities( comm_path ):
    communities = {}
    assigned_nodes = set()
    lines = open(comm_path,"r").readlines()
    for l in lines:
        parts = l.strip().split("\t")
        node_id = int(parts[0])
        assigned_nodes.add(node_id)
        for x in parts[1].split(" "):
            comm_id = int(x)
            if not comm_id in communities:
                communities[comm_id] = set()
            communities[comm_id].add( node_id )
    return communities

In [4]:
g = nx.read_edgelist(data_inpath, nodetype=int)
print("Network: %d nodes, %d edges" % (g.number_of_nodes(), g.number_of_edges() ) )

Network: 1005 nodes, 16706 edges


In [5]:
ground_communities = read_communities(comm_inpath)
print("Read %d ground truth communities" % len(ground_communities))

Read 42 ground truth communities


In [6]:
ground_sizes = [ len(ground_communities[comm_id]) for comm_id in ground_communities ]
sorted(ground_sizes)[::-1]

[109,
 92,
 65,
 61,
 55,
 51,
 49,
 39,
 35,
 32,
 29,
 29,
 28,
 27,
 26,
 25,
 25,
 22,
 19,
 18,
 15,
 14,
 13,
 13,
 13,
 12,
 10,
 10,
 9,
 9,
 8,
 8,
 6,
 6,
 5,
 4,
 4,
 3,
 3,
 2,
 1,
 1]

In [7]:
ground_partition = np.zeros(g.number_of_nodes())
for comm_id in ground_communities:
    comm = ground_communities[comm_id]
    for node_id in comm:
        ground_partition[node_id] = comm_id

### Community Finding

Just for demonstration purposes, find communities in graph using Clauset-Newman-Moore greedy modularity maximization.

In [8]:
communities = list(community.greedy_modularity_communities(g))
k = len(communities)
print("Found %d communities" % k )

Found 44 communities


In [9]:
sizes = [ len(comm) for comm in communities ]
sorted(sizes)[::-1]

[368,
 321,
 103,
 89,
 58,
 9,
 9,
 9,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

Apply NMI validation:

In [10]:
from sklearn.metrics import normalized_mutual_info_score

In [11]:
partition = np.zeros(g.number_of_nodes())
for comm_id, comm in enumerate(communities):
    for node_id in comm:
        partition[node_id] = comm_id

In [12]:
nmi = normalized_mutual_info_score(ground_partition, partition, average_method='arithmetic' )
print("NMI score=%.3f" % nmi)

NMI score=0.478
